# From Recipe to UMF

This notebook walks through calculating UMF from a glaze recipe. It assumes familiarity with [UMF calculations](umf_calculations.ipynb).

A **recipe** lists materials and amounts. Each material contributes oxides. To get UMF:

1. Know each material's oxide analysis
2. Combine them according to the recipe
3. Calculate UMF from the total

In [ ]:
from utils import read_materials, read_recipes, format_umf_table

## Materials

Each material has an **oxide analysis** - the weight percentage of each oxide it contributes.

**LOI (Loss on Ignition)** is weight lost during firing, typically as CO₂ or H₂O. For example, whiting (CaCO₃) is ~56% CaO and ~44% LOI - the calcium carbonate decomposes, releasing CO₂.

In [2]:
materials = read_materials("""
materials:
  custer_feldspar:
    name: Custer Feldspar
    loi: 0.3
    analysis:
      SiO2: 68.5
      Al2O3: 18.2
      K2O: 10.0
      Na2O: 3.0

  silica:
    name: Silica
    analysis:
      SiO2: 100.0

  whiting:
    name: Whiting
    loi: 44.0
    analysis:
      CaO: 56.0

  epk:
    name: EPK Kaolin
    loi: 14.0
    analysis:
      SiO2: 46.0
      Al2O3: 38.0
      Fe2O3: 1.0
      TiO2: 1.0
""")

## Recipe

In [3]:
recipes = read_recipes("""
recipes:
  cone10_clear:
    name: Cone 10 Clear
    materials:
      custer_feldspar: 40
      silica: 30
      whiting: 20
      epk: 10
""")

recipe = recipes['cone10_clear']['materials']

## Batch to Oxide Analysis

Each material contributes oxides proportionally:

$$\text{oxide contribution} = \text{parts} \times \frac{\text{analysis \%}}{100}$$

In [4]:
batch_weight = sum(recipe.values())

total_oxides = {}
total_loi = 0

for mat_id, parts in recipe.items():
    mat = materials[mat_id]
    total_loi += parts * mat.get('loi', 0) / 100
    for oxide, pct in mat['analysis'].items():
        total_oxides[oxide] = total_oxides.get(oxide, 0) + parts * pct / 100

fired_weight = sum(total_oxides.values())

print(f"Batch weight:  {batch_weight:5.1f}")
print(f"Lost to LOI:   {total_loi:5.1f}")
print(f"Fired weight:  {fired_weight:5.1f}")

Batch weight:  100.0
Lost to LOI:    10.3
Fired weight:   89.7


In [5]:
# Normalize to 100%
oxide_analysis = {ox: val / fired_weight * 100 for ox, val in total_oxides.items()}

for ox, pct in oxide_analysis.items():
    print(f"{ox:8} {pct:6.2f}%")
print(f"{'Total':8} {sum(oxide_analysis.values()):6.2f}%")

SiO2      69.13%
Al2O3     12.36%
K2O        4.46%
Na2O       1.34%
CaO       12.49%
Fe2O3      0.11%
TiO2       0.11%
Total    100.00%


## UMF

Convert oxide weight % to UMF (see [UMF calculations](umf_calculations.ipynb)):

In [6]:
molecular_weight = {
    "SiO2": 60.085, "Al2O3": 101.961, "TiO2": 79.866,
    "Li2O": 29.881, "Na2O": 61.979, "K2O": 94.196,
    "MgO": 40.304, "CaO": 56.077, "SrO": 103.619, "BaO": 153.326,
    "ZnO": 81.379, "Fe2O3": 159.688,
}

flux_oxides = ["Li2O", "Na2O", "K2O", "CaO", "MgO", "SrO", "BaO", "ZnO"]

moles = {ox: pct / molecular_weight[ox] for ox, pct in oxide_analysis.items()}
flux_total = sum(moles[ox] for ox in moles if ox in flux_oxides)
umf = {ox: mol / flux_total for ox, mol in moles.items()}

print(format_umf_table(umf, flux_oxides=[ox for ox in flux_oxides if ox in umf]))

Flux:
  Na2O     0.074
  K2O      0.162
  CaO      0.764
  TOTAL    1.000

Other:
  SiO2     3.945
  Al2O3    0.415
  Fe2O3    0.002
  TiO2     0.005


In [7]:
f"SiO2:Al2O3 = {umf['SiO2'] / umf['Al2O3']:.1f}"

'SiO2:Al2O3 = 9.5'